<a href="https://colab.research.google.com/github/CatharinaColleoni/Projeto_Imersao_Alura_Google/blob/main/Query_embedding_in_GeminiAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o SDK do Google

In [5]:
!pip install -q -U google-generativeai

Importações e configurações iniciais

In [6]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('secret_key')
genai.configure(api_key=api_key)

Listar os modelos disponíveis

In [43]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Exemplo de embedding

In [44]:
title = 'The next generation of AI for developers and Google Workspace'
sample_text = ('title = The next generation of AI for developers and Google Workspace'
'\n'
'Full article: \n'
'\n'
'Gemini API & Google AI Studio: an approachable way to explore and prototype with generative AI applications')
embeddings = genai.embed_content(model='models/text-embedding-004',
                                 content=sample_text,
                                 title=title,
                                 task_type='RETRIEVAL_DOCUMENT')
print(embeddings)

{'embedding': [-0.0031897835, -0.0052453326, -0.058816623, -0.00560664, 0.0010388537, 0.04006002, 0.045939364, 0.036578447, -0.042328935, 0.010275192, -0.028677668, 0.013036877, -0.007015399, 0.004825209, -0.013332411, -0.05612247, 0.033416655, 0.004958111, -0.11696105, 0.06605344, -0.020422267, -0.024192825, -0.098142855, -0.01008983, -0.034524817, -0.0087237265, 0.070916094, 0.03765901, 0.022345992, 0.04462867, 0.013642219, 0.0400666, 0.035932425, -0.040860724, -0.01609851, -0.014898463, 0.00835251, -0.017605372, 0.06938587, 0.0015521526, -0.018527502, 0.016878126, 0.0047371024, 0.1253245, -0.022138529, 0.010671266, -0.00026616742, 0.072411925, -0.056505553, 0.016515108, 0.088081166, 0.02184043, -0.06837984, 0.027091336, -0.005243058, 0.0016734967, -0.063790776, -0.0013671123, 0.085975446, 0.030249558, -0.026268141, 0.00304715, -0.057820406, 0.033636685, -0.02364974, -0.014796319, -0.017651109, 0.020296331, -0.04604354, 0.025728425, 0.00894521, -0.016452096, -0.04149153, 0.026223063,

Listagem de documentos que serão buscados

In [46]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [52]:
df = pd.DataFrame(documents)
df.colums = ['title', 'content']
df

<ipython-input-52-2e4dd4f373cc>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.colums = ['title', 'content']


,title,content
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [53]:
model = 'models/text-embedding-004'

In [58]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type='RETRIEVAL_DOCUMENT')['embedding']

In [59]:
df['Embeddings'] = df.apply(lambda row: embed_fn(row['title'], row['content']), axis=1)
df

,title,content,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[0.016963858, 0.0074500735, -0.031256273, -0.0..."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.008353342, 0.029304754, -0.04979913, -0.038..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[0.026820987, 0.01214682, -0.021441665, 0.0134..."


Gerar embedding para pergunta e buscar no dataframe a resposta ideal

In [67]:
def search_and_generate_query(query, df, model):
  query_embedding = genai.embed_content(model=model,
                                 content=query,
                                 task_type='RETRIEVAL_QUERY')
  dot_products = np.dot(np.stack(df['Embeddings']), query_embedding['embedding'])

  index = np.argmax(dot_products)
  return df.iloc[index]['content']

In [68]:
query = 'How do I change gears on my Google car?'

patch = search_and_generate_query(query, df, model)
print(patch)

Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.
